# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

# Principal Agent model for forsikring I

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The agent is a consumer whos utility depends on consumption $c$ given by the utility function $v(c)$.  
The agents consumption depends on income $M$ and whether or not he experiences "accidents". 

If the agent experiences accidents he loses a portion of his income denoted by $L$

with probability $\pi$ he experiences an accident and gets: 

$$
c = M - L
$$

with probability $(1-\pi)$ he does not experience accidents and gets: 

$$
c = M
$$

The principal is an insurance company, that sells insurances with a price $\Gamma$ and payment of $K$;

The agent pays the price and following scenarios can happen;

with probability of $\pi$ the agent experiences an accident and loses $L$, but gets the insurance payment of $K$, Thus

$$
c = M - \Gamma - L + K
$$

with probability 1-$\pi$ the agent does not experience accidents and gets: 

$$
c = M - \Gamma 
$$

The agent has two options,

To sign the insurance contract with the principal, or to choose the outside option denoted by $\overline{v}$

$$
\overline{v} = \pi * v(M-L) + (1-\pi) * v(M)
$$

we assume the agent maximizes utility and we assume $v' > 0$ and $v'' < 0$,
which means that the agent likes to consume and would like to avoid risks. 

We also assume that the principal maximizes expected utility. 

We can now write up our principal agent problem, where $\Gamma - K$ is profit for the principal without accidents and $\Gamma$ is the profit without accidents.
$$
\max_{\Gamma, K} \pi * (\Gamma - K) + (1-\pi) * \Gamma\\

s.t. \\


\pi * v(M - \Gamma - L + K) + (1 - \pi) * v(M - \Gamma) \underline{>} \overline{v}
$$ 

The contraint ensures that the agents utility by accepting the insurance contract is atleast as big as the utility he gets from choosing the outside option, when the function is maximized. 

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

We start by defining $c_1 = M - \Gamma$ as consumption with out accident and $c_2 = M - \Gamma - L + K$ as consumption with accident.

Note that if we choose a particular $\Gamma$ and $K$ we can find $c_1$ and $c_2$ directly, and vice versa if we choose a particular $c_1$ and $c_2$.

There by we can expresse the principal-Agent problem as following, 
$$
\max_{c_1,c_2} M - \pi \cdot L - (1-\pi)c_1 - \pi c_2 \\
s.t. \\
\pi \cdot v(c_2) + (1-\pi) \cdot v(c_1) \geq \overline{v} 
$$

In [2]:
from modelproject import define_symbols, calculate_derivatives, solve_equations

# Definer symbolerne
L, M, pi, c_1, c_2, lam, v_overline, v = define_symbols()

# Beregn derivaterne og opret ligninger
principal, condition, lagrange, Lc_1, Lc_2, Llam, v_diff_c1, v_diff_c2 = calculate_derivatives(L, M, pi, c_1, c_2, lam, v_overline, v)

#passer 

1. We see that the second order condition must be binding
$$
\pi \cdot v(c_2) + (1-\pi) \cdot v(c_1) = \overline{v} 
$$
2. and then we solve the problem using lagrange.

In [3]:
print('L = ')
display(lagrange)

L = 


-L*pi + M - c_1*(1 - pi) - c_2*pi + lambda*(-\overline{v} + pi*v(c_2) + (1 - pi)*v(c_1))

Find the first order derivatives

In [4]:
display(Lc_1)
display(Lc_2)
display(Llam)

Eq(0, lambda*(1 - pi)*Derivative(v(c_1), c_1) + pi - 1)

Eq(0, lambda*pi*Derivative(v(c_2), c_2) - pi)

Eq(0, -\overline{v} + pi*v(c_2) + (1 - pi)*v(c_1))

We solve for $\frac{d}{dc_1}v(c_1)$ and $\frac{d}{dc_2}v(c_2)$

In [5]:
# Solve the system of equations
solution = solve_equations(Lc_1, Lc_2, Llam, v_diff_c1, v_diff_c2)

# check for a solution
if isinstance(solution, dict):
    for var, sol in solution.items():
        eq = sm.Eq(var, sol)
        display(eq)
elif isinstance(solution, list) and solution:
    for sol in solution:
        display(sol)
else:
    print("No solutions found.")

Eq(Derivative(v(c_1), c_1), 1/lambda)

Eq(Derivative(v(c_2), c_2), 1/lambda)

Meaning that optimal contract includes that $c_1 = c_2$. Since $c_1 = M - \Gamma$ can we write $c_2 = M - \Gamma - L + K \Leftrightarrow c_2 = c_1 - L + K \Leftrightarrow L = c_1 - c_2 + K$ which means that $L = K$.

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [ ]:
def parameter():
    M = 5
    gamma = []
    K = []
    L = 2
    par.c1 = M - gamma
    par.c2 = M - gamma - L + K 
    pi = 0.3
    rho = 0.95


def agent_utility(c,rho): #we definere a utility function which is CRRA according to your definition of v
    return c**(1 - rho) / (1 - rho)

def outside_obs(M, L, pi): #expected_consumption
    """Beregner det forventede forbrug givet indkomst M, tab L og sandsynligheden pi for tab."""
    return pi * agent_utility(M-L,rho) + (1 - pi) * agent_utility(M,rho)

In [54]:
def parameter(K=0.5):
    M = 5
    # Assume gamma and K are defined as scalar values here for now
    gamma = 2  # Example value, should be defined appropriately
    #K = []      # Example value, should be defined appropriately
    L = 2
    pi = 0.3
    rho = 0.95

    # Compute consumptions
    c1 = M - gamma
    c2 = M - gamma - L + K

    return M, L, pi, rho, c1, c2

def principal_problem():
    return M - pi * L - (1-pi) * c1 - pi * c2 

def utility(c, rho):
    return c**(1 - rho) / (1 - rho)

def constraint(c1,c2,rho):
    pi * utility(c2,rho) + (1-pi) * utility(c1,rho)

def outside_option(M, L, pi, rho):
    """Calculate expected utility given income M, loss L, and the probability pi of loss."""
    return -(pi * agent_utility(M - L, rho) + (1 - pi) * agent_utility(M, rho))

initial_guess_K = [2]

result = optimize.minimize(principal_problem, initial_guess_K, method='BFGS')
print(result)


# Example usage:
#M, L, pi, rho, c1, c2 = parameter()  # Initialize parameters
#utility_c1 = agent_utility(c1, rho)  # Utility of consumption c1
#utility_c2 = agent_utility(c2, rho)  # Utility of consumption c2
#expected_util = expected_consumption(M, L, pi, rho)  # Expected utility

#print("Utility of c1:", utility_c1)
#print("Utility of c2:", utility_c2)
#print("Expected utility:", expected_util)

TypeError: principal_problem() takes 0 positional arguments but 1 was given

In [52]:
from scipy import optimize

def parameter(K=0.2):
    gamma=2
    M = 5
    L = 2
    pi = 0.3
    rho = 0.95
    c1 = M - gamma
    c2 = M - gamma - L + K
    return M, L, pi, rho, c1, c2

def principal_problem(K, M, L, pi, c1):
    c2 = M - gamma - L + K
    return -(M - pi * L - (1-pi) * c1 - pi * c2)  # Negative for maximization

def utility(c, rho):
    if c <= 0:
        return float('-inf')  # Avoid log of zero or negative numbers if rho == 1
    return c**(1 - rho) / (1 - rho)

def constraint(c1, c2, pi, rho):
    return pi * utility(c2, rho) + (1-pi) * utility(c1, rho)

def optimize_principal():
    M, L, pi, rho, c1, _ = parameter()
    initial_guess_K = [0]  # Assuming K starts from 0
    result = optimize.minimize(principal_problem, initial_guess_K, args=(M, L, pi, c1), method='BFGS')
    return result

result = optimize_principal()
print("Optimization result:", result)

NameError: name 'gamma' is not defined

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 